In [1]:
%run ../notebooks/00_setup_paths.ipynb

utils/ folder added to Python import path
project_root:     C:\Users\akaas\crime-projectMain
raw_dir:          C:\Users\akaas\crime-projectMain\data
parquet_dir:      C:\Users\akaas\crime-projectMain\data_parquet
processed_dir:    C:\Users\akaas\crime-projectMain\data_processed
models_dir:       C:\Users\akaas\crime-projectMain\models
logs_dir:         C:\Users\akaas\crime-projectMain\logs
utils_dir:        C:\Users\akaas\crime-projectMain\utils


In [2]:
from spark_init import init_spark

spark = init_spark("CrimeProject_Phase1")
spark

Spark Initialized: CrimeProject_Phase1


In [3]:
import zipfile
import os
from pathlib import Path
import shutil

parquet_dir.mkdir(exist_ok=True)

for zip_file in raw_dir.glob("*.zip"):
    segment_name = zip_file.stem
    extract_to = parquet_dir / segment_name

    if extract_to.exists() and any(extract_to.iterdir()):
        print(f"Already extracted {zip_file.name}.")
        continue
        
    extract_to.mkdir(exist_ok=True)
    print(f"Extracting {zip_file.name} - {extract_to}")

    with zipfile.ZipFile(zip_file, "r") as z:
        z.extractall(extract_to)

print("All zip files extracted to data_parquet/")

Already extracted administrative_segment.zip.
Already extracted arrestee_segment.zip.
Already extracted batch_header.zip.
Already extracted group_b_arrest_report_segment.zip.
Already extracted offender_segment.zip.
Already extracted offense_segment.zip.
Already extracted property_segment.zip.
Already extracted victim_segment.zip.
Already extracted window_arrestee_segment.zip.
Already extracted window_exceptional_clearance_segment.zip.
Already extracted window_recovered_property_segment.zip.
All zip files extracted to data_parquet/


In [4]:
import pandas as pd

segment_files = {}
for segment_folder in parquet_dir.iterdir():
    if segment_folder.is_dir():
        files = list(segment_folder.glob("*.parquet"))
        segment_files[segment_folder.name] = files

pd.DataFrame({
    "segment": list(segment_files.keys()),
    "num_files": [len(files) for files in segment_files.values()]
})

,segment,num_files
0,administrative_segment,11
1,arrestee_segment,11
2,batch_header,1
3,group_b_arrest_report_segment,11
4,offender_segment,11
5,offense_segment,11
6,property_segment,11
7,victim_segment,11
8,window_arrestee_segment,9
9,window_exceptional_clearance_segment,9


In [5]:
def load_parquet_segment(segment_name):
    folder = parquet_dir / segment_name
    files = list(folder.glob("*.parquet"))
    
    print(f"Loading {segment_name}: {len(files)} files")
    df = spark.read.parquet(*[str(f) for f in files])
    return df

In [6]:
segment_dfs = {}

for segment in segment_files.keys():
    print(f"\n===== LOADING {segment} =====")
    df = load_parquet_segment(segment)
    df.printSchema()
    segment_dfs[segment] = df


===== LOADING administrative_segment =====
Loading administrative_segment: 11 files
root
 |-- ori: string (nullable = true)
 |-- year: double (nullable = true)
 |-- state: string (nullable = true)
 |-- state_abb: string (nullable = true)
 |-- incident_number: string (nullable = true)
 |-- incident_date: string (nullable = true)
 |-- report_date_indicator: string (nullable = true)
 |-- incident_date_hour: string (nullable = true)
 |-- total_offense_segments: double (nullable = true)
 |-- total_victim_segments: double (nullable = true)
 |-- total_offender_segments: double (nullable = true)
 |-- total_arrestee_segments: double (nullable = true)
 |-- city_submissions: string (nullable = true)
 |-- cleared_exceptionally: string (nullable = true)
 |-- exceptional_clearance_date: string (nullable = true)
 |-- unique_incident_id: string (nullable = true)


===== LOADING arrestee_segment =====
Loading arrestee_segment: 11 files
root
 |-- ori: string (nullable = true)
 |-- year: double (nullabl

In [7]:
for segment, df in segment_dfs.items():
    print(f"\n=== SCHEMA: {segment} ===")
    df.printSchema()


=== SCHEMA: administrative_segment ===
root
 |-- ori: string (nullable = true)
 |-- year: double (nullable = true)
 |-- state: string (nullable = true)
 |-- state_abb: string (nullable = true)
 |-- incident_number: string (nullable = true)
 |-- incident_date: string (nullable = true)
 |-- report_date_indicator: string (nullable = true)
 |-- incident_date_hour: string (nullable = true)
 |-- total_offense_segments: double (nullable = true)
 |-- total_victim_segments: double (nullable = true)
 |-- total_offender_segments: double (nullable = true)
 |-- total_arrestee_segments: double (nullable = true)
 |-- city_submissions: string (nullable = true)
 |-- cleared_exceptionally: string (nullable = true)
 |-- exceptional_clearance_date: string (nullable = true)
 |-- unique_incident_id: string (nullable = true)


=== SCHEMA: arrestee_segment ===
root
 |-- ori: string (nullable = true)
 |-- year: double (nullable = true)
 |-- state: string (nullable = true)
 |-- state_abb: string (nullable = tr

In [8]:
for segment, df in segment_dfs.items():
    print(f"\n--- SAMPLE ROWS: {segment} ---")
    df.show(5, truncate=False)


--- SAMPLE ROWS: administrative_segment ---
+---------+------+-------+---------+---------------+-------------+---------------------+-----------------------------+----------------------+---------------------+-----------------------+-----------------------+----------------+------------------------------------------------------------------------------+--------------------------+---------------------------+
|ori      |year  |state  |state_abb|incident_number|incident_date|report_date_indicator|incident_date_hour           |total_offense_segments|total_victim_segments|total_offender_segments|total_arrestee_segments|city_submissions|cleared_exceptionally                                                         |exceptional_clearance_date|unique_incident_id         |
+---------+------+-------+---------+---------------+-------------+---------------------+-----------------------------+----------------------+---------------------+-----------------------+-----------------------+----------------+-

Py4JJavaError: An error occurred while calling o79.showString.
: org.apache.spark.SparkException: [FAILED_READ_FILE.PARQUET_COLUMN_DATA_TYPE_MISMATCH] Encountered error while reading file file:///C:/Users/akaas/crime-projectMain/data_parquet/window_arrestee_segment/nibrs_window_arrestee_segment_2014.parquet. Data type mismatches when reading Parquet column [arrestee_weapon_2]. Expected Spark type string, actual Parquet type DOUBLE. SQLSTATE: KD001
	at org.apache.spark.sql.errors.QueryExecutionErrors$.parquetColumnDataTypeMismatchError(QueryExecutionErrors.scala:847)
	at org.apache.spark.sql.execution.datasources.v2.FileDataSourceV2$.attachFilePath(FileDataSourceV2.scala:138)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:142)
	at org.apache.spark.sql.execution.FileSourceScanExec$$anon$1.hasNext(DataSourceScanExec.scala:695)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.columnartorow_next

In [9]:
segment_counts = {
    segment: df.count()
    for segment, df in segment_dfs.items()
}

segment_counts

{'administrative_segment': 88046163,
 'arrestee_segment': 25905626,
 'batch_header': 704767,
 'group_b_arrest_report_segment': 22659484,
 'offender_segment': 99209713,
 'offense_segment': 99673851,
 'property_segment': 101459391,
 'victim_segment': 99303264,
 'window_arrestee_segment': 415977,
 'window_exceptional_clearance_segment': 86584,
 'window_recovered_property_segment': 63582}

In [10]:
for segment, df in segment_dfs.items():
    print(segment, "→", "unique_incident_id" in df.columns)

administrative_segment → True
arrestee_segment → True
batch_header → False
group_b_arrest_report_segment → False
offender_segment → True
offense_segment → True
property_segment → True
victim_segment → True
window_arrestee_segment → True
window_exceptional_clearance_segment → True
window_recovered_property_segment → True


In [11]:
summary_df = pd.DataFrame({
    "segment": list(segment_counts.keys()),
    "rows": list(segment_counts.values()),
    "num_files": [len(segment_files[s]) for s in segment_counts.keys()]
})

summary_path = processed_dir / "phase1_summary.csv"

if summary_path.exists():
    os.remove(summary_path)

summary_df.to_csv(summary_path, index=False)
summary_df

,segment,rows,num_files
0,administrative_segment,88046163,11
1,arrestee_segment,25905626,11
2,batch_header,704767,1
3,group_b_arrest_report_segment,22659484,11
4,offender_segment,99209713,11
5,offense_segment,99673851,11
6,property_segment,101459391,11
7,victim_segment,99303264,11
8,window_arrestee_segment,415977,9
9,window_exceptional_clearance_segment,86584,9
